In [5]:
import os
import random
import math
import io
import glob
from pathlib import Path

import numpy as np
import cv2 as cv

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [22]:
TEMPL_DIR = "./screenshots/hp"
REAL_TEST_DIR = "./hp_test"
OUT_DIR = "./hp_model_out"

IMG_SIZE = 32
TRAIN_STEPS = 8000
VAL_STEPS   = 800
SYNTH_TEST_STEPS = 1500
BATCH_SIZE = 256
EPOCHS = 12
SEED = 42
NUM_CLASSES = 4

In [23]:
os.makedirs(OUT_DIR, exist_ok=True)
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [26]:
def load_class_templates(root):
    rootp = Path(root)
    if not rootp.exists():
        raise FileNotFoundError(f"not found: {root}")
    classes = {}
    for c in range(NUM_CLASSES):
        paths = sorted(list(rootp.glob(f"{c}*.png")))
        imgs = []
        for p in paths:
            img = cv.imread(str(p), cv.IMREAD_UNCHANGED)
            if img is None:
                continue
            if img.ndim == 3 and img.shape[2] == 4:
                img = cv.cvtColor(img, cv.COLOR_BGRA2BGR)
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            imgs.append(img)
        if not imgs:
            raise FileNotFoundError(f"Missing templates for class {c}.")
        classes[c] = imgs
    return classes

class_templates = load_class_templates(TEMPL_DIR)

In [27]:
def letterbox_gray_np(rgb):
    g = cv.cvtColor(rgb, cv.COLOR_RGB2GRAY)
    h, w = g.shape[:2]
    scale = min(IMG_SIZE / h, IMG_SIZE / w)
    nh, nw = max(1,int(round(h*scale))), max(1,int(round(w*scale)))
    resized = cv.resize(g, (nw, nh), interpolation=cv.INTER_AREA)
    canvas = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)
    top = (IMG_SIZE - nh)//2
    left = (IMG_SIZE - nw)//2
    canvas[top:top+nh, left:left+nw] = resized.astype(np.float32)/255.0
    return canvas

template_banks = {c: [letterbox_gray_np(im) for im in ims] for c, ims in class_templates.items()}

In [33]:
augment_geo = keras.Sequential([
    layers.RandomRotation(factor=0.033, fill_mode="constant", seed=SEED),
    layers.RandomZoom(height_factor=(-0.15, 0.15), width_factor=(-0.15,0.15), fill_mode="constant", seed=SEED),
    layers.RandomTranslation(height_factor=0.06, width_factor=0.06, fill_mode="constant", seed=SEED),
], name="geom_aug")

random_contrast = layers.RandomContrast(0.25, seed=SEED)

In [17]:
@tf.function
def random_brightness(x):
    delta = tf.random.uniform(shape=(tf.shape(x)[0], 1, 1, 1), minval=-0.08, maxval=0.08, seed=SEED)
    return tf.clip_by_value(x + delta, 0., 1.)

@tf.function
def brighten_flash(x):
    strength = tf.random.uniform((tf.shape(x)[0], 1, 1, 1), 0.10, 0.40, seed=SEED+1)
    return tf.clip_by_value(x*(1.0-strength) + strength, 0., 1.)

def gaussian_kernel(k=7, sigma=1.2):
    ax = tf.range(-k//2 + 1, k//2 + 1, dtype=tf.float32)
    xx, yy = tf.meshgrid(ax, ax)
    kernel = tf.exp(-(xx**2 + yy**2)/(2.0*sigma**2))
    kernel /= tf.reduce_sum(kernel)
    kernel = tf.reshape(kernel, (k, k, 1, 1))
    return kernel

G_KERNEL = gaussian_kernel(7, 1.2)

@tf.function
def glow_bloom(x):
    blur = tf.nn.depthwise_conv2d(x, tf.repeat(G_KERNEL, repeats=1, axis=2), strides=[1, 1, 1, 1], padding="SAME")
    out = tf.clip_by_value(x + 0.5*blur, 0., 1.)
    mn = tf.reduce_min(out, axis=[1, 2, 3], keepdims=True)
    mx = tf.reduce_max(out, axis=[1, 2, 3], keepdims=True)
    out = tf.where(mx>mn, (out-mn)/(mx-mn), out)
    return out

In [30]:
def make_synth_dataset(n_steps_per_class, batch_size, augment=True, shuffle=True, seed=SEED):
    labels = np.repeat(np.arange(NUM_CLASSES), n_steps_per_class)
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(labels)

    ds = tf.data.Dataset.from_tensor_slices(labels)

    def gen_sample(c):
        bank = template_banks[int(c.numpy())]
        idx = np.random.randint(0, len(bank))
        img = bank[idx]
        return img.astype(np.float32), np.int32(c.numpy())

    def py_map(c):
        img, lab = tf.py_function(gen_sample, [c], [tf.float32, tf.int32])
        img.set_shape((IMG_SIZE, IMG_SIZE))
        lab.set_shape(())
        img = tf.expand_dims(img, -1)
        return img, lab

    ds = ds.map(py_map, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        def aug(img, lab):
            return img, lab
        ds = ds.map(aug, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(4096, seed=seed, reshuffle_each_iteration=True)

    ds = ds.batch(batch_size, drop_remainder=True)

    if augment:
        def aug_batched(img, lab):
            x = img
            x = augment_geo(x)
            x = random_contrast(x)
            x = random_brightness(x)
            if tf.random.uniform(()) < 0.6:
                x = brighten_flash(x)
            if tf.random.uniform(()) < 0.7:
                x = glow_bloom(x)
            return x, lab
        ds = ds.map(aug_batched, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_synth_dataset(
    n_steps_per_class=TRAIN_STEPS, batch_size=BATCH_SIZE, augment=True, shuffle=True, seed=SEED
)
val_ds = make_synth_dataset(
    n_steps_per_class=VAL_STEPS, batch_size=BATCH_SIZE, augment=True, shuffle=False, seed=SEED+1
)


test_ds = make_synth_dataset(
    n_steps_per_class=SYNTH_TEST_STEPS, batch_size=512, augment=True, shuffle=False, seed=SEED+2025
)
print(f"Using synthetic test set ({SYNTH_TEST_STEPS*NUM_CLASSES} samples).")


Using SYNTHETIC fixed-seed test set (6000 samples).


In [31]:
def build_model():
    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))
    x = layers.Conv2D(8, 3, padding="same", activation="relu")(inp)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(16, 3, padding="same", activation="relu")(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = layers.MaxPool2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(32, activation="relu")(x)
    out = layers.Dense(NUM_CLASSES, activation=None)(x)
    model = keras.Model(inp, out)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
    )
    return model

model = build_model()
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 32, 32, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 32, 32, 8)           │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 16, 16, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 16, 16, 16)          │           1,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 8, 8, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 32)            │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 4, 4, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │          16,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,436 (87.64 KB)

 Trainable params: 22,436 (87.64 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
steps_per_epoch = max(1, (TRAIN_STEPS * NUM_CLASSES) // BATCH_SIZE)
val_steps = max(1, (VAL_STEPS * NUM_CLASSES) // BATCH_SIZE)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=val_steps,
    verbose=1
)


Epoch 1/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 143ms/step - acc: 0.4491 - loss: 1.2322 - val_acc: 0.8164 - val_loss: 0.5617
Epoch 2/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 132ms/step - acc: 0.8708 - loss: 0.4686 - val_acc: 0.9336 - val_loss: 0.2493
Epoch 3/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 132ms/step - acc: 0.9578 - loss: 0.1618 - val_acc: 0.9899 - val_loss: 0.0779
Epoch 4/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - acc: 0.9855 - loss: 0.0656 - val_acc: 0.9951 - val_loss: 0.0356
Epoch 5/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 132ms/step - acc: 0.9941 - loss: 0.0334 - val_acc: 0.9980 - val_loss: 0.0211
Epoch 6/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 145ms/step - acc: 0.9912 - loss: 0.0319 - val_acc: 0.9880 - val_loss: 0.0489
Epoch 7/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - acc: 0.9930 - loss: 0.0287 - val_acc: 0.9997 - val_loss: 0.0074
Epoch 8/12
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 137ms/step - acc: 0.9990 - loss: 0.0081 - val_acc: 0.9993 - val_loss: 0.0070
Epoch 9/12
125/125 ━━━━━━━━━━━━━

In [ ]:
test_metrics = model.evaluate(test_ds, verbose=1)
print(f"\nTest loss: {test_metrics[0]:.4f},  test acc: {test_metrics[1]:.4f}")

In [19]:
model.save(str(Path(OUT_DIR)/"hp_tf_model.keras"))
print("SavedModel exported to:", saved_path)

SavedModel exported to: hp_model_out\saved_model


In [21]:
onnx_path = Path("hp_model_out/hp_tf.onnx")
spec = (tf.TensorSpec((None, IMG_SIZE, IMG_SIZE, 1), tf.float32, name="input"),)

model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13,
    output_path=onnx_path.as_posix(),
)
print("ONNX saved to:", onnx_path)

rewriter <function rewrite_constant_fold at 0x00000239EB6EA2A0>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


ONNX saved to: hp_model_out\hp_tf.onnx
